# Conectando ao Banco

In [1]:
import psycopg2
import csv

connection = psycopg2.connect(
    dbname='projeto_bd',
    user='postgres',
    password='2805caique',
    host='localhost',
    port='5432'
)

cursor = connection.cursor()

In [ ]:
def imprimir_tabela_ascii(dados, cabecalhos):
    """
    Imprime uma tabela ASCII a partir de uma lista de tuplas e cabeçalhos.

    :param dados: Lista de tuplas, onde cada tupla representa uma linha.
    :param cabecalhos: Lista de strings com os nomes das colunas.
    """
    if not dados or not cabecalhos:
        print("Nenhum dado para mostrar.")
        return

    num_colunas = len(cabecalhos)

    # Descobre o maior número de casas decimais entre os floats
    max_decimais = 0
    for linha in dados:
        for valor in linha:
            if isinstance(valor, float):
                str_valor = str(valor)
                if '.' in str_valor:
                    decimais = len(str_valor.split('.')[1])
                    max_decimais = max(max_decimais, decimais)

    # Cria nova matriz de dados com floats formatados
    dados_formatados = []
    for linha in dados:
        nova_linha = []
        for valor in linha:
            if isinstance(valor, float):
                novo_valor = f"{valor:.{max_decimais}f}"
            else:
                novo_valor = str(valor)
            nova_linha.append(novo_valor)
        dados_formatados.append(tuple(nova_linha))

    # Calcula largura de cada coluna
    larguras = []
    for i in range(num_colunas):
        maior = max(len(str(linha[i])) for linha in dados_formatados)
        larguras.append(max(maior, len(cabecalhos[i])) + 2)  # +2 para espaçamento

    # Imprime cabeçalho
    linha_cabecalho = " | ".join(cabecalhos[i].ljust(larguras[i]) for i in range(num_colunas))
    print(linha_cabecalho)
    print("-" * len(linha_cabecalho))

    # Imprime os dados formatados
    for linha in dados_formatados:
        linha_formatada = " | ".join(
            f"{linha[i].rjust(larguras[i]) if linha[i].replace('.', '', 1).isdigit() else linha[i].ljust(larguras[i])}"
            for i in range(num_colunas)
        )
        print(linha_formatada)


# Top 10 países que mais emitiram CO2 desde o início do milênio 

In [6]:
q = """
    SELECT 
        p.nome AS nome_pais,
        sum((e.emissao * pa.qntd_habitantes)) AS "Total de toneladas emitidas"
    FROM 
        public.emissao_gee e
    JOIN 
        public.populacao_anual pa 
        ON e.ano = pa.ano AND e.fk_gee_pais = pa.fk_populacao_pais
    JOIN 
        public.pais p 
        ON e.fk_gee_pais = p.id_pais
    WHERE e.ano >= 2000
    GROUP BY nome_pais
    ORDER BY 
        "Total de toneladas emitidas" desc
"""

cursor.execute(q)

resultados = cursor.fetchmany(10)

In [9]:
cabecalhos = ['Pais', 'Toneladas de CO2 emitidas desde 2000']
imprimir_tabela_ascii(resultados, cabecalhos)

Pais                             | Toneladas de CO2 emitidas desde 2000  
-------------------------------------------------------------------------
China, People's Republic of      |                    243547219984.869995
United States of America         |                    154023784095.767120
India, Republic of               |                     71280949285.556015
Brazil, Federative Republic of   |                     60888640878.083687
Russian Federation               |                     54071990157.028496
Indonesia, Republic of           |                     40520463476.537857
Japan                            |                     29935607473.227959
Germany, Federal Republic of     |                     21616726778.602295
Canada                           |                     20338714102.405571
Iran, Islamic Republic of        |                     19250677906.647640


In [11]:
cabecalhos = ['Pais', 'Toneladas de CO2 emitidas desde 2000']
imprimir_tabela_ascii(resultados, cabecalhos)

Pais                             | Toneladas de CO2 emitidas desde 2000  
-------------------------------------------------------------------------
China, People's Republic of      |                        243547219984.87
United States of America         |                     154023784095.76712
India, Republic of               |                      71280949285.55602
Brazil, Federative Republic of   |                      60888640878.08369
Russian Federation               |                     54071990157.028496
Indonesia, Republic of           |                      40520463476.53786
Japan                            |                      29935607473.22796
Germany, Federal Republic of     |                     21616726778.602295
Canada                           |                      20338714102.40557
Iran, Islamic Republic of        |                      19250677906.64764
